In [23]:
noise = ['balanced', 'bias', 'flip']
level = ['0.1','0.3']
dataset = ['income', 'compas', 'adult', 'synthetic_20']
shifts = [ '0.5_1.5','0.25_1.1','1_2']

In [3]:
import os
import shutil

In [24]:
base_path = '/Users/sauravanchlia/Fair_ML/Projects/debias_shift/Debias/data/covariate_shifted_data/semi_synthetic'
for sh in shifts:
    for n in noise:
        for l in level:
            for ds in dataset:
                # for f in ['train_ids.csv', 'test_ids.csv', 'valid_ids.csv']:
                try:
                    new_path = os.path.join(base_path, sh, n, l, ds)
                    old_path = os.path.join(base_path, sh, ds, n, l)
                    if os.path.exists(old_path):
                        shutil.copytree(old_path, new_path)
                    else:
                        print(f'{old_path=} does not exist')
                except FileExistsError:
                    continue
                except FileNotFoundError as fe:
                    print(f'{fe=} , {old_path=}')
                    continue
